# Линейная регрессия: прогноз оклада по описанию вакансии

Данное задание основано на материалах лекций по линейной регрессии и посвящено предсказанию оклада, исходя из описания вакансии.

## Вы научитесь:
- использовать линейную регрессию
- применять линейную регрессию к текстовым данным

## Введение
Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

## Реализация в Scikit-Learn

Для извлечения TF-IDF-признаков из текстов воспользуйтесь классом `sklearn.feature_extraction.text.TfidfVectorizer`.

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе `sklearn.linear_model.Ridge`.

Обратите внимание, что признаки `LocationNormalized` и `ContractTime` являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке — кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют one-hot-кодированием. Воспользуйтесь им, чтобы перекодировать признаки `LocationNormalized` и `ContractTime`. Он уже реализован в классе `sklearn.feature_extraction.DictVectorizer`.
#### Пример использования:

```python
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
```

Вам понадобится производить замену пропущенных значений на специальные строковые величины (например, `'nan'`). Для этого подходит следующий код:
```python
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)
```

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge

## 1
Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла `salary-train.csv` (либо его заархивированную версию `salary-train.zip`).

In [2]:
data = pd.read_csv('salary-train.csv')
test = pd.read_csv('salary-test-mini.csv')

In [3]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [4]:
test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [5]:
print('Data length: ', len(data), '\nTest length: ', len(test))

Data length:  60000 
Test length:  2


In [6]:
cols = data.keys()
print(data.keys())
print(data.loc[0])

Index(['FullDescription', 'LocationNormalized', 'ContractTime',
       'SalaryNormalized'],
      dtype='object')
FullDescription       International Sales Manager London ****k  ****...
LocationNormalized                                               London
ContractTime                                                  permanent
SalaryNormalized                                                  33000
Name: 0, dtype: object


## 2
Проведите предобработку:
- Приведите тексты к нижнему регистру (`text.lower()`).
- Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке `text` подходит следующий вызов: `re.sub('[^a-zA-Z0-9]', ' ', text)`. Также можно воспользоваться методом `replace` у `DataFrame`, чтобы сразу преобразовать все тексты:

```python
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
```

- Примените `TfidfVectorizer` для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр `min_df` у `TfidfVectorizer`).
- Замените пропуски в столбцах `LocationNormalized` и `ContractTime` на специальную строку `'nan'`. Код для этого был приведен выше.
- Примените `DictVectorizer` для получения one-hot-кодирования признаков `LocationNormalized` и `ContractTime`.
- Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией `scipy.sparse.hstack`.

In [7]:
def transform_data(data):
    # present data correctly
    data.FullDescription = data.FullDescription.replace('[^a-zA-Z0-9]', ' ', regex=True).str.lower()
    return data

In [8]:
data = transform_data(data)
test = transform_data(test)

In [9]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


In [11]:
vectorizer = TfidfVectorizer(min_df=5)

# count tf-idf for description
tf_idf = vectorizer.fit_transform(data.FullDescription)

In [12]:
print(tf_idf[:5])
print('\n\n', tf_idf.shape)

  (0, 10690)	0.199670497595
  (0, 18017)	0.313912346218
  (0, 12439)	0.0491059414075
  (0, 12142)	0.0615447891631
  (0, 21382)	0.0514772476117
  (0, 4349)	0.0450094078629
  (0, 6080)	0.0910718154989
  (0, 12553)	0.223398307339
  (0, 15185)	0.105064934474
  (0, 9624)	0.126501341378
  (0, 1192)	0.0438009173709
  (0, 4403)	0.0211108535457
  (0, 1270)	0.257559860181
  (0, 14410)	0.0282512348983
  (0, 14296)	0.046515126365
  (0, 20595)	0.136843386937
  (0, 11716)	0.0536857053059
  (0, 10249)	0.11720974528
  (0, 7387)	0.0520100925297
  (0, 12546)	0.0313371059483
  (0, 1518)	0.043116715981
  (0, 18372)	0.0341052693559
  (0, 7489)	0.0424821050072
  (0, 3095)	0.0590871311897
  (0, 5946)	0.0232477153105
  :	:
  (4, 21026)	0.0474905767737
  (4, 16130)	0.0527969038139
  (4, 2723)	0.182294808389
  (4, 18113)	0.0942423839764
  (4, 15699)	0.0688158808603
  (4, 61)	0.0881786407467
  (4, 5059)	0.0713049286218
  (4, 12625)	0.102076294343
  (4, 11762)	0.093414395616
  (4, 19536)	0.0901931261697
  (4, 793

In [13]:
# fill empty values in location and contract time
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [14]:
one_hot = DictVectorizer()

# one hot transformation for text fields
oh = one_hot.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [15]:
print(oh[:5])
print('\n\n', oh.shape)

  (0, 2)	1.0
  (0, 957)	1.0
  (1, 2)	1.0
  (1, 957)	1.0
  (2, 2)	1.0
  (2, 1392)	1.0
  (3, 2)	1.0
  (3, 471)	1.0
  (4, 1)	1.0
  (4, 1495)	1.0


 (60000, 1766)


In [16]:
X = hstack([tf_idf, oh])
y = data['SalaryNormalized']

In [17]:
print(X.shape)
print('\n\n', y[:5])

(60000, 24627)


 0    33000
1    50000
2    40000
3    22500
4    20355
Name: SalaryNormalized, dtype: int64


## 3
Обучите гребневую регрессию с параметрами `alpha=1` и `random_state=241`.
Целевая переменная записана в столбце `SalaryNormalized`.

In [18]:
clf = Ridge(alpha=1, random_state=241)
clf.fit(X, y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

## 4
Постройте прогнозы для двух примеров из файла `salary-test-mini.csv`.
Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [19]:
tf_idf_test = vectorizer.transform(test.FullDescription)
oh_test = one_hot.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([tf_idf_test, oh_test])

res = clf.predict(X_test)
print(res)

f = open('submission.txt', 'w')
f.write(str(res)[1:-1])
f.close()

[ 56555.61500155  37188.32442618]
